In [1]:
import os

In [2]:
%pwd

'/Users/kunal/Developer/DL Project/Chicken-Disease-Classification-Using-Fecal-Image/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/kunal/Developer/DL Project/Chicken-Disease-Classification-Using-Fecal-Image'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    # Initializing parameters from config.yaml file
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    # Initializing parameters from params.yaml file
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    # This function will load the updated base model
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    # Generating training and validation data from my image directory present in artifacts folder
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # performing some augmentation techniques as we have given AUGMENTATION = True in params.yaml file
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    # After training it will save the model
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator, # training data I am giving
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator # validation data I am giving
        )

        # After training I am saving the model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator() # here I am converting the data into training and validation
    training.train()
    
except Exception as e:
    raise e

[2025-02-28 17:16:29,004: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-28 17:16:29,005: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-28 17:16:29,006: INFO: common: created directory at: artifacts]
[2025-02-28 17:16:29,007: INFO: common: created directory at: artifacts/training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/300
19/19 [==============================] - 31s 2s/step - loss: 12.3395 - accuracy: 0.5541 - val_loss: 5.6762 - val_accuracy: 0.6094
Epoch 2/300
19/19 [==============================] - 33s 2s/step - loss: 8.8762 - accuracy: 0.6081 - val_loss: 3.0324 - val_accuracy: 0.6875
Epoch 3/300
19/19 [==============================] - 34s 2s/step - loss: 6.1838 - accuracy: 0.6757 - val_loss: 6.1053 - val_accuracy: 0.5469
Epoch 4/300
19/19 [==============================] - 35s 2s/step - loss: 3.6968 - accuracy: 0.7905 - val_loss: 0.9767 - val_accuracy: 0.9219
Epoch 5/300
19/19 [=====

/opt/homebrew/anaconda3/envs/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
